In [ ]:
import json
import requests
import pandas as pd

In [ ]:
#FINANCITAL_API = 'https://www.kavak.com/api/vip-ui/mx/calculator/468814?upfront-amount=116499'
FINANCITAL_API = 'https://www.kavak.com/api/vip-ui/mx/calculator'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9'
}

In [ ]:
def get_minimum_upfront_amount(price):
    return float(price) * 0.16

In [ ]:
def get_financial_data(auto_id, headers, price):
    api_url = f"{FINANCITAL_API}/{auto_id}"
    upfront_amount = int(get_minimum_upfront_amount(price))
    query_params = {
        'upfront-amount': upfront_amount
    }

    response = requests.get(api_url, headers=headers, params=query_params)
    return response

In [ ]:
"""
with open('../data/dataset_autos.jsonl', 'r', encoding='utf-8') as f:
    line = f.readline()
    data = json.loads(line)
    print(data['id'], data['slug'], data['price'])
    referer = data['slug'] + "?" + "id=" + data['id']
    headers_copy = HEADERS.copy()
    headers_copy['Referer'] = referer
    financial_data = get_financial_data(data['id'], headers_copy, data['price'])
    print(financial_data.text)
"""

In [ ]:
referer = "https://www.kavak.com/mx/usado/mercedes_benz-clase_glc-20_glc_300_off_road_auto_4wd-suv-2019" + "?" + "id=" + str(452889)
headers_copy = HEADERS.copy()
headers_copy['Referer'] = referer
data = get_financial_data(452889, headers_copy, 338999)

In [ ]:
data_json = data.json()
paymentPlans = data_json['offers']['paymentPlan']['paymentOptions']['UPFRONT_VALUE']
inputData = data_json['offers']['inputData']

In [ ]:
def get_plan_info(plan):
    mensualidades = plan['installments']
    enganche = plan['value']
    tasa = plan['rate']
    if plan['insurance']:
        seguro = plan['insurance']['installmentAmount']
    else: seguro = None
    return mensualidades, enganche, tasa, seguro

def get_upfront_info(inputData):
    value = inputData['value']
    min_upfront_value = inputData['min']
    max_upfront_value = inputData['max']
    return value, min_upfront_value, max_upfront_value

def extract_financial_info(auto_id, paymentPlans, inputData, precio):
    data_plan_list = []
    value, min_upfront_value, max_upfront_value = get_upfront_info(inputData)

    for plan in paymentPlans:
        plazo, mensualidad, tasa, seguro = get_plan_info(plan)
        data_plan_list.append({
            'ID_Auto':auto_id,
            'Precio':precio,
            'Plazo':plazo,
            'Mensualidad':mensualidad, 
            'Tasa':tasa, 
            'Seguro':seguro,
            'Enganche_Simulado':value, 
            'Enganche_Min':min_upfront_value, 
            'Enganche_Max':max_upfront_value
            })
    
    df_data_plan = pd.DataFrame(data_plan_list)

    return df_data_plan



In [ ]:
df_planes = extract_financial_info(452889, paymentPlans, inputData, 338999)
print(df_planes)